# 데이터 전처리


- 데이터를 분석에 사용할 때 성능이 더 좋게 나오도록 데이터를 수정하거나 형태를 변형하는 작업
- data preprocessing

<img src="https://raw.githubusercontent.com/data-labs/image/main/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202021-01-07%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%204.33.49.png?raw=1" width=500  align='left'>  



## 데이터 전처리 종류

- 데이터 클리닝 - 결측치 처리, 틀린값 처리
- 이상치 처리 - 이상치 탐지 및 제거
- 데이터 변환 - 로그 변환, 카테고리 인코딩 등
- 스케일링 - 여러 변수 값의 범위를 동등한 조건으로 맞추는 것
  - 표준 스케일링, min-max 스케일링

# import

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

%config InlineBackend.figure_format = 'retina'

# 데이터 생성
- 임의의 키, 몸무게 데이터 생성
- 평균 170cm, 65kg, 표준편차 4cm, 5kg

In [19]:
## 100명의 키(170,4), 몸무게(65,5) 데이터 생성 

N = 100
height = 4*np.random.randn(N).round(2) + 170
weight = 5*np.random.randn(N).round(2) + 65
df = pd.DataFrame({"height": height, "weight": weight})
df[:5]

,height,weight
0,164.84,60.55
1,170.44,64.25
2,166.00,67.70
3,162.32,70.60
4,172.00,74.40


In [20]:
## 임의로 결측치 삽입
# np.nan는 결측치를 나타내는 것임 (not a number)

df['height'][2] = np.nan
df['weight'][3] = np.nan
df[:5]

,height,weight
0,164.84,60.55
1,170.44,64.25
2,NaN,67.70
3,162.32,NaN
4,172.00,74.40


# 데이터 클리닝

##  결측치 처리
- 결측치를 처리하는 방법은 크게 세 가지가 있다
  - 결측치가 포함된 샘플(행)을 버린다
  - 결측치를 적절한 값으로 대체한다
  - 결측치 처리를 다음 분석 단계로 넘긴다. 즉, 결측치를 그대로 둔다
 
##  틀린값 처리
- 틀린값을 처리하는 방법도 결측치를 처리하는 방법과 같이 세가지이다
  - 틀린 값이 포함된 샘플(행)을 버린다
  - 틀린 값을 적절한 값으로 대체한다
  - 틀린값 처리를 다음 분석 단계로 넘긴다. 즉, 틀린값을 그대로 둔다

## 결측치 처리 실습

- 결측치 확인은 np.isnull() 사용
- 결측치 치환은 np.fillna() 사용

In [21]:
## 컬럼별 결측치 총 개수 보기

df.isnull().sum()

height    1
weight    1
dtype: int64

### 결측치가 있는 행 삭제

In [22]:
## 결측치가 하나라도 있는 행(샘플) 삭제하기
# 아래에서 행이 2 감소한 것을 알 수 있다

df2 = df.dropna()
print(df2.shape)
df2[:5]

(98, 2)


,height,weight
0,164.84,60.55
1,170.44,64.25
4,172.00,74.40
5,170.96,64.20
6,171.84,62.80


### 결측치 대체

In [23]:
## 결측치 대체하기 (키는 170으로 몸무게는 평균치로 대체)
# inplace=True는 실행결과를 원본 데이터에 즉시 반영하라는 뜻임

df3 = df.copy()
df3['height'].fillna(170, inplace=True)
df3['weight'].fillna(df3['weight'].mean(), inplace=True)
print(df3.shape)
df3[:5]

(100, 2)


,height,weight
0,164.84,60.550000
1,170.44,64.250000
2,170.00,67.700000
3,162.32,65.905556
4,172.00,74.400000


In [24]:
## 결측치를 바로 앞의 값으로 대체하는 경우 (forward fill)

df.fillna(method="ffill")[:5]

,height,weight
0,164.84,60.55
1,170.44,64.25
2,170.44,67.70
3,162.32,67.70
4,172.00,74.40


In [25]:
## 결측치를 바로 뒤의 값으로 대체하는 경우 (backward fill)

df.fillna(method="bfill")[:5]

,height,weight
0,164.84,60.55
1,170.44,64.25
2,162.32,67.70
3,162.32,74.40
4,172.00,74.40


# 이상치 처리
- 이상치 처리의 목적은 두가지 이다
 - 1) 이상치를 제거하여 데이터 분석의 정확도를 높이려는 경우
 - 2) 이상치를 찾는 것 자체가 목적인 경우 (outlier detection)  
 
- 아래는 모두 같은 상관계수를 같는 데이터 셋이다

<img src="https://raw.githubusercontent.com/data-labs/image/main/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202021-01-03%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%201.47.02.png?raw=1" width=400 align='left'>

## 이상치 탐지 방법
- 단변수의 경우
  - 예를 들어 표준편차가 3 이상 떨어진 샘플을 찾는다
  - boxplot()을 보면 각 단변수에 대해서 이상치를 파악하기 쉽다
- 여러 변수를 동시에 고려해야 하는 경우
  - 주성분 분석(PCA)이나 차원 축소를 하여 멀리 떨어져 있는 샘플을 찾는다
  - 이상치를 레이블(목적변수)로 지정하고 지도학습을 하여 이상치를 찾는 모델을 만든다

- PCA는 샘플을 저차원 특성공간에서 (보통 2차원) 샘플간의 분포를 파악할 때 자주 사용된다 
<img src="https://github.com/StillWork/image/blob/main/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202021-05-30%20%E1%84%8B%E1%85%A9%E1%84%8C%E1%85%A5%E1%86%AB%209.31.41.png?raw=1" align='left'  width=400>

# 데이터 변환

- 카테고리 인코딩
  - 국적, 성별, 계절 등 카테고리를 구분하는 변수는 일반적으로 원 핫 인코딩으로 표현한다
  - 연속형 변수를 카테고리 타입으로 변환하여 사용하기도 한다(10대, 20대 등 연령대, 실력 등급, 옷 사이즈 등)
- 로그변환: x 대신 log(x)를 사용
- 역수변환: 1/x 사용
- 기타 변환
  - 다차항(polynomial) 변환: x^2 나 x*y를 사용
  - 비율값 사용:  x/y
  - 새로운 값 정의: BMI = (몸무게)/(키**2)

## 카테고리 인코딩
- 카테고리 변수는 대부분 원핫인코딩(one hot encoding)을 하여 0,1 의 숫자로 표현한다
- pd.get_dummies()를 사용한다

In [26]:
## 카테고리 인코딩을 위한 데이터 생성 (임의로 국적 컬럼을 추가)

human = df3[['height','weight']][:5]
human['nation'] = ['한국', '중국', '일본', '한국', '중국']
human

,height,weight,nation
0,164.84,60.550000,한국
1,170.44,64.250000,중국
2,170.00,67.700000,일본
3,162.32,65.905556,한국
4,172.00,74.400000,중국


In [27]:
## humna 데이터프레임에서 nation 컬럼을 원핫 인코딩하는 방법

pd.get_dummies(human, ['nation'])

,height,weight,nation_일본,nation_중국,nation_한국
0,164.84,60.550000,0,0,1
1,170.44,64.250000,0,1,0
2,170.00,67.700000,1,0,0
3,162.32,65.905556,0,0,1
4,172.00,74.400000,0,1,0


# 스케일링
- 여러 변수 값의 범위가 서로 다른 경우 데이터 분석에서 동일한 비중으로 반영하기 위해서는 동일한 범위를 갖도록 변환해야 한다 (예: 과목 성적 100점 만점으로 환산)
- 표준 스케일링을 가장 많이 사용한다 (평균이 0, 표준편차가 1이 되도록 변환)
- 값의 최소치와 최대치가 정해진 경우는 min-max 스케링일도 자주 사용된다 (최소치를 0으로, 최대치를 1로)

## 표준 스케일링


<img src="https://raw.githubusercontent.com/data-labs/image/main/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202021-01-05%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%208.52.14.png?raw=1" width=140  align='left'>

In [28]:
## 키, 몸무게 데이터를 최소-최대 및 표준 스케일링한 결과 보기

from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = StandardScaler()
minmax = MinMaxScaler()

df3[['h_sc','w_sc']]=scaler.fit_transform(df3[['height','weight']])
df3[['h_mima','w_mima']]=minmax.fit_transform(df3[['height','weight']])
df3[:5]

,height,weight,h_sc,w_sc,h_mima,w_mima
0,164.84,60.550000,-1.505220,-1.060538,0.279826,0.270541
1,170.44,64.250000,0.107104,-0.327843,0.583514,0.418838
2,170.00,67.700000,-0.019578,0.355346,0.559653,0.557114
3,162.32,65.905556,-2.230766,0.000000,0.143167,0.485193
4,172.00,74.400000,0.556252,1.682120,0.668113,0.825651


In [29]:
# 표준 스케일링에 사용한 평균치와 표준편차 보기 - 각 컬럼별로 보여준다
print(scaler.mean_, scaler.scale_)

[170.068       65.90555556] [3.47324632 5.04984598]


- 스케일링 이전의 원래 값으로 환원하는 방법
- inverse_transform을 사용한다

In [30]:
## 스케일링 이전의 값으로 환원하기

scaler.inverse_transform(df3[["h_sc","w_sc"]])[:5]

array([[164.84      ,  60.55      ],
       [170.44      ,  64.25      ],
       [170.        ,  67.7       ],
       [162.32      ,  65.90555556],
       [172.        ,  74.4       ]])

## (중요) fit_transform()과 transform()을 구분

- 최초에 훈련데이터로 스케일링을 할때는 fit_transform()를 사용한다
  - fit_transform() 함수는 fit()과 transform() 두개의 함수를 한번에 수행하는 함수이다
  -  fit()에서는 평균, 표준편차 등을 계산하는 과정이고 transform()에서 입력 값의 변환작업을 수행한다
- 향후 테스트 데이터 등 새로운 데이터를 스케일링할 때는 fit_transform()를 다시 수행하면 안되고, 앞에서 만든 scaler 객체가 제공하는 scaler.transform()을 사용해야 한다
  - 앞에서 만든 스케일링 조건(평균, 표준편차, 최대값, 최소값 등)과 동일한 방식으로 스케일링을 수행해야 하므로
  - fit_transform()를 실행하면 스케일링 조건(평균, 표준편차, 최대값, 최소값 등)을 새로운 데이터를 가지고 다시 계산한다

# (실습) 원핫 인코딩

- 위의 human 데이터 프레임에는 국적이 3가지가 있는데 여기에서 5번째 행(샘플)의 국적을 '인도'로 변경하여 총 4가지의 국적을 구분하는 원핫인코딩을 수행하시오